In [1]:
from default_settings import *

In [2]:
import tensorflow_probability as tfp

In [3]:
tfd = tfp.distributions
tfb = tfp.bijectors

# Bayesian A/B Testing

A/B testing is a statistical design pattern for determining the difference of effectiveness between two different treatments. For example, a pharmaceutical company is interested in the effectiveness of drug A vs drug B. The company will test drug A on some fraction of their trials, and drug B on the other fraction (this fraction is often 1/2, but we will relax this assumption). After performing enough trials, the in-house statisticians sift through the data to determine which drug yielded better results.

## A simple case
As this is a hacker book, we'll continue with the web-dev example. For the moment, we will focus on the analysis of site A only. Assume that there is some true $0 \lt p_A \lt 1$ probability that users who, upon shown site A, eventually purchase from the site. This is the true effectiveness of site A. Currently, this quantity is unknown to us.

Suppose site A was shown to $N$ people, and $n$ people purchased from the site. One might conclude hastily that $p_A = \frac{n}{N}$. Unfortunately, the observed frequency $\frac{n}{N}$ does not necessarily equal $p_A$ -- there is a difference between the observed frequency and the true frequency of an event. The true frequency can be interpreted as the probability of an event occurring. For example, the true frequency of rolling a 1 on a 6-sided die is $\frac{1}{6}$. Knowing the true frequency of events like:

- fraction of users who make purchases,
- frequency of social attributes,
- percent of internet users with cats etc.
are common requests we ask of Nature. Unfortunately, often Nature hides the true frequency from us and we must *infer* it from observed data.

The *observed frequency* is then the frequency we observe: say rolling the die 100 times you may observe 20 rolls of 1. The observed frequency, 0.2, differs from the true frequency, $\frac{1}{6}$. We can use Bayesian statistics to infer probable values of the true frequency using an appropriate prior and observed data.

With respect to our A/B example, we are interested in using what we know, $N$ (the total trials administered) and $n$ (the number of conversions), to estimate what $p_A$, the true frequency of buyers, might be.

To setup a Bayesian model, we need to assign prior distributions to our unknown quantities. A priori, what do we think $p_A$ might be? For this example, we have no strong conviction about $p_A$, so for now, let's assume $p_A$ is uniform over $[0,1]$:

In [4]:
reset_sess()

# The parameters are the bounds of the Uniform.
p = tfd.Uniform(low=0, high=1., name='p')

Had we had stronger beliefs, we could have expressed them in the prior above.

For this example, consider $p_A = 0.05$, and $N = 1500$ users shown site A, and we will simulate whether the user made a purchase or not. To simulate this from $N$ trials, we will use a Bernoulli distribution: if  $X\ \sim \text{Ber}(p)$, then $X$ is 1 with probability $p$ and 0 with probability $1 - p$. Of course, in practice we do not know $p_A$, but we will use it here to simulate the data. We can assume then that we can use the following generative model:

$$\begin{align*}
p \,\sim \text{Uniform}[\text{low}=0,\text{high}=1) \\
X\,\sim \text{Bernoulli}(\text{prob}=p) \\
\text{for }  i \,= 1\ldots N:\text{# Users}  \\
 X_i\,\sim \text{Bernoulli}(p_i)
\end{align*}$$

In [5]:
reset_sess()

# set constants
prob_true = 0.05 # remember, this is unknown
N = 1500

# Sample N Bernoulli random variables from Ber(0.05)
# Each random variable has a 0.05 chance of being a 1.
# This is the data-generation step

occurrences = tfd.Bernoulli(probs=prob_true).sample(sample_shape=N, seed=6.45)

[
    occurrences_,
    occurrences_sum_,
    occurrences_mean_,
] = evaluate([
    occurrences,
    tf.reduce_sum(occurrences),
    tf.reduce_mean(tf.to_float(occurrences))
])

print('Array of {} occurrences:'.format(N), occurrences_)
print('(Remember: Python treats True == 1, and False == 0)')
print('Sum of (True == 1) Occurrences:', occurrences_sum_)

Array of 1500 occurrences: [0 0 0 ... 0 0 0]
(Remember: Python treats True == 1, and False == 0)
Sum of (True == 1) Occurrences: 76


The observed frequency is:

In [6]:
# occurrences.mean is equal to n/N.
print("what is the observed frequency in Group A? {}".format(occurrences_mean_))
print("Does this equal to the true frequency? {}".format(occurrences_mean_ == prob_true))

what is the observed frequency in Group A? 0.05066666752099991
Does this equal to the true frequency? False


We combine our Bernoulli distribution and our observed occurrences into a log probability function based on the two.

In [7]:
def joint_log_prob(occurrences, prob_A):
    """Joint probability optimization function.
    
    Parameters
    ----------
    occurences: array-like, binary values 
        Observed frequency
    prob_A: scalar
        Probability of a 1 appearing.
        
    Returns
    -------
    Joint log probability optimization function.
    """
    rv_prob_A = tfd.Uniform(low=0, high=1.)
    
    rv_occurrences = tfd.Bernoulli(probs=prob_A)
    
    return rv_prob_A.log_prob(prob_A) + tf.reduce_sum(rv_occurrences.log_prob(occurrences))

The goal of probabilistic inference is to find model parameters that may explain data you have observed. TFP performs probabilistic inference by evaluating the model parameters using a `joint_log_prob` function. The arguments to `joint_log_prob` are data and model parameters - for the model defined in the `joint_log_prob` function itself. The function returns the log of the joint probability that the model parameterized as such generated the observed data per the input arguments.

All `joint_log_prob` functions have a common structure:
1. The function takes a set of **inputs** to evaluate. Each input is either an observed value or a model parameter.
2. The `joint_log_prob` function uses probability distribution to define a **model** for evaluating the inputs. These distributions measure the likelihood of the input values. (By convention, the distribution that measures the likelihood of the variable `foo` will be named `rv_foo` to note that it is a random variable). We use two types of distributions in `joint_log_prob` functions:
    1. **Prior distributions** measure the likelihood of input values. A prior distribution never depends on an input value each prior distribution measures the likelihood of a single input value. Each unknown variable - one that has not been observed directly - needs a corresponding prior. Beliefs about which values could be reasonable determine the prior distribution. Choosing a prior can be tricky.
    2. **Conditional distributions** measure the likelihood of an input value given other input values. Typically, the conditional distributions return the likelihood of observed data given the current guess of parameters in the model, *p*(observed_data | model_parameters).
    3. Finally we calculate and return the **joint log probability** of the inputs. The joint log probability is the sum of the log probabilities from all the prior and conditional distributions. (We take the sum of log probabilities instead of multiplying the probabilities directly for reasons of numerical stability: floating point numers in computers cannot represent the very small values needed to calculate the joint log probability unless they are in log space). The sum of probabilities is actually an unnormalized density; although the total sum of probabilities over all possible inputs might not sum to one, the sum of probabiltiies is proportional to the true probability density. The proportional distribution is sufficient to estimate the distribution of likely inputs. 
    
Let's map these tems onto the code above. In this example, the input values are the observed values in `occurrences` and the unknown value for `prob_A`. The `joint_log_prob` takes the current guess for `prob_A` and answers, how likely is the data if `prob_A` is the probability of `occurrences`. The answer depends on two distributions:

1. The prior distribution, `rv_prob_A`, indicates how likely the current value of `prob_A` is by itself.
2. The conditional distribution, `rv_occurrences`, indicates the likelihood of `occurrences` if `prob_A` were the probability for the Bernoulli distribution. 

The sum of the log of these probabiities is the joint log probability.

The `joint_log_prob` is particularly useful in conjunction with the `tfp.mcmc` module. Markov chain Monte Carlo (MCMC) algorithms proceed by making educated guesses about the unknown input values and computing what the likelihood of this set of arguments is. By repeating this process many times, MCMC builds a distribution of likely parameters. Constructing this distribution is the goal of probabilistic inference.

In [9]:
number_of_steps = 48000
burnin = 25000
leapfrog_steps = 2

# Set the chain's start state.
initial_chain_state = [
    tf.reduce_mean(tf.to_float(occurrences))
    * tf.ones([], dtype=tf.float32, name='init_prob_A')
]

# Since HMC operates over unconstrained space, we need to transform the 
# samples so they live in real space.
unconstraining_bijectors = [
    tfp.bijectors.Identity()
]

# Define a closure over our joint_log_prob
# The closure marks it so the HMC doesn't try to change the 'occurrences' but instead determines the distributions
# of other parameters that might generate the 'occurrences' we observed.
unnormalized_posterior_log_prob = lambda *args: joint_log_prob(occurrences, *args)

# Initialize the step size. (It will be automatically adapted.)
with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    step_size = tf.get_variable(name='step_size',
                               initializer=tf.constant(0.5, dtype=tf.float32),
                               trainable=False,
                               use_resource=True)
# Defining the HMC
hmc = tfp.mcmc.TransformedTransitionKernel(
    inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=unnormalized_posterior_log_prob,
        num_leapfrog_steps=leapfrog_steps,
        step_size=step_size,
        step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(),
        state_gradients_are_stopped=True),
    bijector=unconstraining_bijectors)

# Sampling from the chain.
[
    posterior_prob_A
], kernel_results = tfp.mcmc.sample_chain(
    num_results=number_of_steps,
    num_burnin_steps=burnin,
    current_state=initial_chain_state,
    kernel=hmc)

# Initialize any created variables.
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

### Execute the TF graph to sample from the posterior

In [10]:
evaluate(init_g)
evaluate(init_l)
[
    posterior_prob_A,
    kernel_results_,
] = evaluate([
    posterior_prob_A,
    kernel_results,
])

print('acceptance rate: {}'.format(
    kernel_results_.inner_results.is_accepted.mean()))

burned_prob_A_trace_ = posterior_prob_A[burnin:]

acceptance rate: 0.6121458333333333
